In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split, cross_val_score

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [2]:
df_train = pd.read_csv('/kaggle/input/iitg-ai-recruitment-2025-beyond-the-box/atlantis_citizens_final.csv')
df_test = pd.read_csv('/kaggle/input/iitg-ai-recruitment-2025-beyond-the-box/test_atlantis_hidden.csv')
df_train.drop(index=df_train[df_train['Wealth_Index']>=21152.0].index, inplace=True)

In [3]:
df_train['Wealth_Index'].quantile(0.9)

np.float64(11956.0)

In [4]:
df_train.shape

(14281, 10)

In [5]:
x_train=df_train.drop(columns=['Occupation', 'Citizen_ID', 'Bio_Hash'])
#y_train=df_train['Occupation']
y_train = df_train['Occupation'].map({'Warrior': 0, 'Merchant': 1, 'Fisher': 2, 'Miner': 3, 'Scribe': 4})

In [6]:
x_test=df_test.drop(columns=['Citizen_ID', 'Bio_Hash'])
#y_test=df_test['Occupation'] -->do this if test data also has this column

#x_test aur x_train mein same features hone chahiye 
#yaha occupation nahi drop kiya kyuki test mein woh hai hi nahi

In [7]:
num_cols = x_train.select_dtypes(include='number').columns
cat_cols = x_train.select_dtypes(exclude='number').columns

In [8]:
num_pipeline=Pipeline(steps=[('Imputer', SimpleImputer(strategy='median'))])
num_pipeline_lr=Pipeline(steps=[('Imputer', SimpleImputer(strategy='median')), ('Scaler', StandardScaler())])

In [9]:
cat_pipeline=Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown="ignore"))])

In [10]:
preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, num_cols), ('cat', cat_pipeline, cat_cols)])
preprocessor_lr = ColumnTransformer(transformers=[('num', num_pipeline_lr, num_cols), ('cat_1', cat_pipeline, cat_cols)])

**Logistic Regression cv**

In [11]:
lr = Pipeline(steps=[('preprocess', preprocessor_lr), ('model', LogisticRegression(random_state=50))])

In [12]:
scores_lr = cross_val_score(lr, x_train, y_train, cv = 5, scoring = "f1_macro")

In [13]:
print(scores_lr.mean())
#drop royal submarine --> 0.6166284115899193
#drop 0.99 quantile --> 0.616826100397273
#drop 0.95 quantile --> 0.6172347929191451
#drop 0.90 quantile --> 0.617433382308022

0.617433382308022


In [14]:
print(scores_lr.std())

0.0066749971911018655


**Decision Tree cv**

In [15]:
#dt = Pipeline(steps=[('preprocess', preprocessor), ('model', DecisionTreeClassifier(random_state=50))])

In [16]:
#scores_dt = cross_val_score(dt, x_train, y_train, cv = 5, scoring = "f1_macro")

In [17]:
#print(scores_dt.mean())

**Random Forest cv**

In [18]:
#rf = Pipeline(steps=[('preprocess', preprocessor), ('model', RandomForestClassifier(random_state=50))])

In [19]:
#scores_rf = cross_val_score(rf, x_train, y_train, cv = 5, scoring = "f1_macro")

In [20]:
#print(scores_rf.mean())

**Gradient Boosting cv**

In [21]:
#gb = Pipeline(steps=[('preprocess', preprocessor), ('model', GradientBoostingClassifier(random_state=50))])

In [22]:
#scores_gb = cross_val_score(gb, x_train, y_train, cv = 5, scoring = "f1_macro")

In [23]:
#print(scores_gb.mean())

In [24]:
#print(scores_gb.std())

**XGBoost cv**

In [25]:
#xg = Pipeline(steps=[('preprocess', preprocessor), ('model', XGBClassifier(random_state=50))])

In [26]:
#scores_xg = cross_val_score(xg, x_train, y_train, cv = 5, scoring = "f1_macro")

In [27]:
#print(scores_xg.mean())

**LightGBM cv**

In [28]:
#lg = Pipeline(steps=[('preprocess', preprocessor), ('model', LGBMClassifier(random_state=50))])

In [29]:
#scores_lg = cross_val_score(lg, x_train, y_train, cv = 5, scoring = "f1_macro")

In [30]:
#print(scores_lg.mean())

**Predicting values with logistic regression**

In [31]:
lr.fit(x_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('Scaler',
                                                                   StandardScaler())]),
                                                  Index(['Wealth_Index', 'House_Size_sq_ft', 'Life_Expectancy'], dtype='object')),
                                                 ('cat_1',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Diet_Type', 'District_Name', 'Vehicle_Owned', 'Work_District'], dtype='object'))])),
                ('model', LogisticRegression(random_state=50))])

In [32]:
y_predict=lr.predict(x_test)

In [33]:
print(y_predict)

[3 4 0 ... 1 3 1]


In [34]:
y_predict = pd.Series(y_predict)


In [35]:
y_predict=pd.DataFrame({"Citizen_ID":df_test["Citizen_ID"], "Occupation":y_predict})

In [36]:
y_predict.to_csv("submission_drop.csv", index=False)